# Hash tables, hash maps, and dictionaries

In [86]:
# Project setup
using Pkg
pkg"activate ."

Activating environment at `~/Dropbox/Projects/Julia Data Structures/Chapter 5/support_files/Project.toml`


## The hash function

In [1]:
function simplehash(val) 
  sum(Int.([c for c::Char in string(val)])) 
end 

simplehash (generic function with 1 method)

In [2]:
simplehash("Adrian")

591

In [3]:
simplehash(:John)

399

In [4]:
simplehash("Caroline")

813

In [5]:
simplehash("Cornelia")

813

### Rehashing

In [6]:
simplehash(simplehash("Cornelia"))

156

## Dict constructors

In [7]:
Dict(:a => "A", :b => "B")

Dict{Symbol,String} with 2 entries:
  :a => "A"
  :b => "B"

In [8]:
(:a => "A") |> typeof

Pair{Symbol,String}

In [9]:
Dict{Any,Any}(:a => "A", :b => "B")

Dict{Any,Any} with 2 entries:
  :a => "A"
  :b => "B"

In [10]:
Dict()

Dict{Any,Any} with 0 entries

In [11]:
Dict{String,Float64}()

Dict{String,Float64} with 0 entries

In [12]:
Dict([("notebook", 2.35), ("pencil", 0.99), ("rubber", 0.35)])

Dict{String,Float64} with 3 entries:
  "notebook" => 2.35
  "pencil"   => 0.99
  "rubber"   => 0.35

In [13]:
Dict(x => repeat(string(x), rand(1:5)) for x in [:a, :b, :c, :d])

Dict{Symbol,String} with 4 entries:
  :a => "a"
  :b => "bbbbb"
  :d => "ddd"
  :c => "cc"

## Getting and setting values

In [14]:
bonuses = Dict("Fred" => 485.12, "Liam" => 112.0, "Sarah" => 418.5)

Dict{String,Float64} with 3 entries:
  "Liam"  => 112.0
  "Sarah" => 418.5
  "Fred"  => 485.12

In [15]:
bonuses["Judy"] = 1_291

1291

In [16]:
bonuses

Dict{String,Float64} with 4 entries:
  "Liam"  => 112.0
  "Sarah" => 418.5
  "Fred"  => 485.12
  "Judy"  => 1291.0

In [17]:
bonuses["Judy"] = "1_291"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [18]:
bonuses["Judy"] = 1_300

1300

In [19]:
bonuses["Sam"]

KeyError: KeyError: key "Sam" not found

In [20]:
haskey(bonuses, "Sam")

false

In [21]:
haskey(bonuses, "Fred")

true

In [22]:
bonuses["NO_BONUS"] = 0

0

In [23]:
sam_bonus = bonuses[getkey(bonuses, "Sam", "NO_BONUS")]

0.0

In [24]:
fred_bonus = bonuses[getkey(bonuses, "Fred", "NO_BONUS")]

485.12

In [25]:
sarah_bonus = get(bonuses, "Sarah", 0)

418.5

In [26]:
get(bonuses, "Sam", 0)

0

In [27]:
bonuses["Sergy"]

KeyError: KeyError: key "Sergy" not found

In [28]:
get!(bonuses, "Sergy", 0)

0.0

In [29]:
bonuses["Sergy"]

0.0

In [30]:
get(bonuses, "Sam") do 
    uppercase("Sam is not up for bonus this year") 
end

"SAM IS NOT UP FOR BONUS THIS YEAR"

## Deleting elements

In [31]:
delete!(bonuses, "Sergy")

Dict{String,Float64} with 5 entries:
  "Liam"     => 112.0
  "Sarah"    => 418.5
  "Fred"     => 485.12
  "Judy"     => 1300.0
  "NO_BONUS" => 0.0

In [32]:
pop!(bonuses, "Liam")

112.0

In [33]:
pop!(bonuses, "Liam")

KeyError: KeyError: key "Liam" not found

In [34]:
pop!(bonuses, "Liam", 0)

0

## Iterating over dictionaries

In [35]:
for bonus in bonuses 
    println(bonus) 
end

"Sarah" => 418.5
"Fred" => 485.12
"Judy" => 1300.0
"NO_BONUS" => 0.0


In [36]:
for (name, bonus) in bonuses 
    println("The bonus for $name is $bonus") 
end

The bonus for Sarah is 418.5
The bonus for Fred is 485.12
The bonus for Judy is 1300.0
The bonus for NO_BONUS is 0.0


In [37]:
keys(bonuses)

Base.KeySet for a Dict{String,Float64} with 4 entries. Keys:
  "Sarah"
  "Fred"
  "Judy"
  "NO_BONUS"

In [38]:
values(bonuses)

Base.ValueIterator for a Dict{String,Float64} with 4 entries. Values:
  418.5
  485.12
  1300.0
  0.0

In [39]:
Symbol <: Any && Int <: Number

true

In [40]:
function userage(x::Any, y::Number) 
    x, y 
end

userage (generic function with 1 method)

In [41]:
userage(:age, 20)

(:age, 20)

In [42]:
function userage(d::Dict{Any,Number}) 
    first(d)[1], first(d)[2] 
end

userage (generic function with 2 methods)

In [43]:
userage(Dict(:age => 20))

MethodError: MethodError: no method matching userage(::Dict{Symbol,Int64})
Closest candidates are:
  userage(::Any, !Matched::Number) at In[40]:2
  userage(!Matched::Dict{Any,Number}) at In[42]:2

In [44]:
userage(Dict{Any,Number}(:age => 20))

(:age, 20)

## Building a custom HashMap implementation

### Constructors

In [45]:
mutable struct HashMap{K,V} 
  storage::Vector{Vector{Pair{K,V}}} 
end

In [46]:
function HashMap{K,V}() where {K,V} 
  Pair{K,V}[] |> HashMap{K,V} 
end

In [47]:
HashMap{Symbol,String}()

HashMap{Symbol,String}(Array{Pair{Symbol,String},1}[])

In [48]:
function HashMap() 
  HashMap{Any,Any}() 
end

HashMap

In [49]:
HashMap()

HashMap{Any,Any}(Array{Pair{Any,Any},1}[])

### Adding the introspection API

In [52]:
function Base.size(hm::HashMap) :: Tuple{Int} 
  size(hm.storage) 
end

In [53]:
function Base.length(hm::HashMap) :: Int 
  length(hm.storage) 
end

In [54]:
function Base.pairs(hm::HashMap) 
  (p for b in hm.storage for p in b) 
end

In [55]:
function Base.keys(hm::HashMap) 
  (p[1] for p in pairs(hm)) 
end

In [56]:
function Base.values(hm::HashMap) 
  (p[2] for p in pairs(hm)) 
end

#### Keys related utilities

In [57]:
function Base.haskey(hm::HashMap{K}, key::K)::Bool where {K} 
  key in keys(hm) 
end

In [58]:
function Base.getkey(hm::HashMap{K}, key::K, default::K)::K where {K} 
  haskey(hm, key) ? key : default 
end 

### Setting values

In [59]:
function index(hm::HashMap{K}, key::K)::Int where {K} 
  length(hm) == 0 && return 1 
  (hash(key) % length(hm)) + 1 
end

index (generic function with 1 method)

In [60]:
function loadfactor(hm::HashMap) :: Float64 
  length(collect(keys(hm))) / length(hm) 
end

loadfactor (generic function with 1 method)

In [61]:
const SHRINK_FACTOR = 0.5 
const EXPAND_FACTOR = 1.5 
 
function rehash!(hm::HashMap{K,V})::HashMap{K,V} where {K,V} 
  lf = loadfactor(hm) 
       
  if isnan(lf)  
    push!(hm.storage, Pair{K,V}[]) # when length of storage array is 0 
    return hm 
  end 
     
  factor = if lf > EXPAND_FACTOR 
    ceil(Int, length(hm) * 2) 
  elseif lf < SHRINK_FACTOR 
    ceil(Int, length(hm) / 2) 
  else 
    return hm 
  end 
             
  data = copy(hm.storage) 
  hm.storage = typeof(hm.storage)() 
                
  for i in 1:factor 
    push!(hm.storage, Pair{K,V}[]) 
  end 
 
  for i in eachindex(data) 
    for (k,v) in data[i] 
      pushvalue(hm, k, v) 
    end 
  end 
             
  hm 
end

rehash! (generic function with 1 method)

In [62]:
function pushvalue(hm::HashMap{K}, key::K, val::V)::V where {K,V} 
  bucket = hm.storage[index(hm, key)] 
 
  for i in 1:length(bucket) 
    if bucket[i][1] == key 
      bucket[i] = Pair{K,V}(key, val) 
      return val 
    end 
  end 
 
  push!(bucket, Pair{K,V}(key, val)) 
     
  val 
end

pushvalue (generic function with 1 method)

In [63]:
function Base.setindex!(hm::HashMap{K,V}, val::V, key::K)::V where {K,V} 
  length(hm) == 0 && rehash!(hm) 
     
  val = pushvalue(hm, key, val) 
     
  rehash!(hm) 
 
  val 
end

### Getting values

In [64]:
function Base.getindex(hm::HashMap{K,V}, key::K)::V where {K,V} 
  bucket = hm.storage[index(hm, key)] 
 
  for i in 1:length(bucket) 
    if bucket[i][1] == key 
      return bucket[i][2] 
    end 
  end 
     
  throw(KeyError(key)) 
end

In [65]:
function Base.get(hm::HashMap{K,V}, key::K, default::V)::V where {K,V} 
  haskey(hm, key) ? hm[key] : default 
end 

In [66]:
function Base.get!(hm::HashMap{K,V}, key::K, default::V)::V where {K,V} 
  haskey(hm, key) && return hm[key] 
 
  hm[key] = default 
end 

### Deleting elements

In [67]:
function Base.delete!(hm::HashMap{K}, key::K)::HashMap{K} where {K} 
  bucket = hm.storage[index(hm, key)] 
 
  for i in 1:length(bucket) 
    if bucket[i][1] == key 
      deleteat!(bucket, i) 
      rehash!(hm) 
             
      return hm 
    end 
  end 
 
  return hm 
end

In [68]:
function Base.pop!(hm::HashMap{K,V}, key::K, default::Union{V,Nothing} = nothing)::V where {K,V} 
  if ! haskey(hm, key) 
    default !== nothing && return default 
    throw(KeyError(key)) 
  end 
 
  val = hm[key] 
  delete!(hm, key) 
 
  val 
end

In [69]:
function Base.empty!(hm::HashMap) :: HashMap 
  empty!(hm.storage) 
     
  hm 
end

### HashMap iteration

In [70]:
function Base.iterate(hm::HashMap{K,V}, state::Union{K,Nothing} = first(keys(hm))) where {K,V} 
  ks = keys(hm) 
 
  idx = findfirst(x -> x == state, ks) 
  idx === nothing && return nothing 
 
  return length(ks) >= idx+1 ? (hm, ks[idx+1]) : nothing 
end

### Trying it out

In [71]:
salaries = HashMap{String,Float64}()

HashMap{String,Float64}(Array{Pair{String,Float64},1}[])

In [72]:
salaries["Bob"] = 42_814.5

42814.5

In [73]:
salaries["Tom"] = 76_012.99

76012.99

In [74]:
pairs(salaries) |> collect

2-element Array{Pair{String,Float64},1}:
 "Tom" => 76012.99
 "Bob" => 42814.5 

In [75]:
haskey(salaries, "Tom")

true

In [76]:
haskey(salaries, "Jane")

false

In [77]:
get!(salaries, "Jane", 135_000.0)

135000.0

In [78]:
pop!(salaries, "Bob")

42814.5

In [79]:
delete!(salaries, "Tom")

HashMap{String,Float64}(Array{Pair{String,Float64},1}[["Jane" => 135000.0], []])

In [80]:
salaries = HashMap{String,Float64}()

HashMap{String,Float64}(Array{Pair{String,Float64},1}[])

In [81]:
names = ["Anna", "Andrew", "Ashton", "Alice", "Armando", "Arnold", "Annie", "Amanda", "Ashley", "Bob", "Bill", "Barney", "Charles", "Cherry", "Chris", "Carmen", "Connor", "Dan", "Daisy", "Don", "Eugene", "Emily", "Eleanor"]

23-element Array{String,1}:
 "Anna"   
 "Andrew" 
 "Ashton" 
 "Alice"  
 "Armando"
 "Arnold" 
 "Annie"  
 "Amanda" 
 "Ashley" 
 "Bob"    
 "Bill"   
 "Barney" 
 "Charles"
 "Cherry" 
 "Chris"  
 "Carmen" 
 "Connor" 
 "Dan"    
 "Daisy"  
 "Don"    
 "Eugene" 
 "Emily"  
 "Eleanor"

In [82]:
for name in names 
   salaries[name] = Float64(rand(15_000:250_000)) 

   println("Load factor: $(loadfactor(salaries))") 
   println("Number of buckets: $(length(salaries))") 
end

Load factor: 1.0
Number of buckets: 1
Load factor: 1.0
Number of buckets: 2
Load factor: 1.5
Number of buckets: 2
Load factor: 1.0
Number of buckets: 4
Load factor: 1.25
Number of buckets: 4
Load factor: 1.5
Number of buckets: 4
Load factor: 0.875
Number of buckets: 8
Load factor: 1.0
Number of buckets: 8
Load factor: 1.125
Number of buckets: 8
Load factor: 1.25
Number of buckets: 8
Load factor: 1.375
Number of buckets: 8
Load factor: 1.5
Number of buckets: 8
Load factor: 0.8125
Number of buckets: 16
Load factor: 0.875
Number of buckets: 16
Load factor: 0.9375
Number of buckets: 16
Load factor: 1.0
Number of buckets: 16
Load factor: 1.0625
Number of buckets: 16
Load factor: 1.125
Number of buckets: 16
Load factor: 1.1875
Number of buckets: 16
Load factor: 1.25
Number of buckets: 16
Load factor: 1.3125
Number of buckets: 16
Load factor: 1.375
Number of buckets: 16
Load factor: 1.4375
Number of buckets: 16


In [83]:
for name in names 
   delete!(salaries, name) 

   println("Load factor: $(loadfactor(salaries))") 
   println("Number of buckets: $(length(salaries))") 
end 

Load factor: 1.375
Number of buckets: 16
Load factor: 1.3125
Number of buckets: 16
Load factor: 1.25
Number of buckets: 16
Load factor: 1.1875
Number of buckets: 16
Load factor: 1.125
Number of buckets: 16
Load factor: 1.0625
Number of buckets: 16
Load factor: 1.0
Number of buckets: 16
Load factor: 0.9375
Number of buckets: 16
Load factor: 0.875
Number of buckets: 16
Load factor: 0.8125
Number of buckets: 16
Load factor: 0.75
Number of buckets: 16
Load factor: 0.6875
Number of buckets: 16
Load factor: 0.625
Number of buckets: 16
Load factor: 0.5625
Number of buckets: 16
Load factor: 0.5
Number of buckets: 16
Load factor: 0.875
Number of buckets: 8
Load factor: 0.75
Number of buckets: 8
Load factor: 0.625
Number of buckets: 8
Load factor: 0.5
Number of buckets: 8
Load factor: 0.75
Number of buckets: 4
Load factor: 0.5
Number of buckets: 4
Load factor: 0.5
Number of buckets: 2
Load factor: 0.0
Number of buckets: 1


## Ordered dictionaries

In [87]:
using Pkg
pkg"add DataStructures"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Dropbox/Projects/Julia Data Structures/Chapter 5/support_files/Project.toml`
  [864edb3b] + DataStructures v0.17.5
  Updating `~/Dropbox/Projects/Julia Data Structures/Chapter 5/support_files/Manifest.toml`
  [864edb3b] + DataStructures v0.17.5
  [bac558e1] + OrderedCollections v1.1.0


In [88]:
using DataStructures

In [89]:
od = OrderedDict{String,Float64}()

OrderedDict{String,Float64} with 0 entries

In [90]:
od["Bob"] = 52_725.15

52725.15

In [91]:
od["Sarah"] = 90_200.

90200.0

In [92]:
34_016.5

34016.5

In [93]:
od

OrderedDict{String,Float64} with 2 entries:
  "Bob"   => 52725.2
  "Sarah" => 90200.0